# Загрузка и обработка файлов

## Скачивание данных, распаковка и запись

In [ ]:
'''
import os
from urllib.request import urlretrieve
import zipfile

# Создание папки data, если не существует
os.makedirs('./data', exist_ok=True)

# Пути к файлам
train_zip_path = './data/train.zip'
valid_zip_path = './data/valid.zip'

# Скачивание архивов
urlretrieve('https://www.dropbox.com/scl/fi/bel6gt6vsb3onahlxvyjc/train_fix.zip?rlkey=q2wscp6wv9j2hbk07y1mbcm54&dl=1', train_zip_path)
urlretrieve('https://www.dropbox.com/scl/fi/cwwblwhvqgwubb8a4xg90/valid.zip?rlkey=mow899lvyawq4wku2m8lfvrh3&dl=1', valid_zip_path)

# Распаковка
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall('./data/train')

with zipfile.ZipFile(valid_zip_path, 'r') as zip_ref:
    zip_ref.extractall('./data/valid')

print("Готово! Архивы скачаны и распакованы в папку ./data")
'''


Готово! Архивы скачаны и распакованы в папку ./data


## Разделим виды бабочек по классам в разные папки

In [6]:
import os
import shutil
from pathlib import Path

def restructure_dataset(source_dir):
    for file_name in os.listdir(source_dir):
        if not file_name.endswith(('.jpg', '.jpeg', '.png')):
            continue

        # Извлекаем имя класса из имени файла (до первой скобки)
        class_name = file_name.split(' (')[0]
        class_dir = os.path.join(source_dir, class_name)

        # Создаем папку, если нужно
        os.makedirs(class_dir, exist_ok=True)

        # Перемещаем файл в папку
        src_path = os.path.join(source_dir, file_name)
        dst_path = os.path.join(class_dir, file_name)
        shutil.move(src_path, dst_path)

# Применяем к train и valid
restructure_dataset('./data/train')
restructure_dataset('./data/valid')

print("✅ Структура папок исправлена")


✅ Структура папок исправлена


# Подготовка данных

In [11]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F

In [4]:
# Аугментации и нормализация
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [8]:
# Загрузка датасета
train_data_full = datasets.ImageFolder(root='./data/train', transform=transform)
valid_data = datasets.ImageFolder(root='./data/valid', transform=transform)

# Разделение тренировочной на обучающую и внутреннюю валидацию
train_size = int(0.8 * len(train_data_full))
val_size = len(train_data_full) - train_size
train_data, train_val_data = random_split(train_data_full, [train_size, val_size])

In [9]:
# DataLoader'ы
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(train_val_data, batch_size=32)
valid_loader = DataLoader(valid_data, batch_size=32)

In [10]:
# Число классов
num_classes = len(train_data_full.classes)
print(f"Всего классов: {num_classes}")

Всего классов: 75


# Работа с моделью

## Создание модели

In [13]:
class ButterflyCNN(nn.Module):
    def __init__(self, num_classes):
        super(ButterflyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) 
        x = self.pool(F.relu(self.conv2(x))) 
        x = self.pool(F.relu(self.conv3(x))) 
        x = x.view(-1, 64 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## Обучение модели

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ButterflyCNN(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Тренировка
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.3f}, Accuracy: {acc:.2f}%")


Epoch 1/5, Loss: 867.766, Accuracy: 10.53%
Epoch 2/5, Loss: 531.868, Accuracy: 39.24%
Epoch 3/5, Loss: 361.023, Accuracy: 56.74%
Epoch 4/5, Loss: 245.505, Accuracy: 69.84%
Epoch 5/5, Loss: 138.975, Accuracy: 82.62%


In [ ]:
torch.save(model.state_dict(), './model/butterfly_model.pth')
print("Модель сохранена")

Модель сохранена


In [17]:
model.load_state_dict(torch.load('./model/butterfly_model.pth'))
model.to(device)

C:\Users\Alexey\AppData\Local\Temp\ipykernel_14340\1610739513.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./model/butterfly_model.p

ButterflyCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=75, bias=True)
)